### Crytocurrencies analyis

#### Objective
Show statistical 'analysis' on
- Random variable 'winning', where X='BTC' is the event where BTC price increase the most in pct compared to other ccys during in interval
- Random varible 'no trade', where X='BTC' is the event where BTC was not traded during an interval

#### Approach
Just explore data by counting for now 


#### Dependencies

In [27]:
# Tech
import os
import sys
import pandas as pd
import numpy as np
import pendulum

module_path = os.path.join('..')
if module_path not in sys.path:
    sys.path.append(module_path)

from src.models import DeltaPoint, Trade    
from src.computed import get_pricepoints_winner_count

In [28]:
# Non tech settings
analysis_interval = 15 # seconds
analysis_period_start = pendulum.parse('2017-08-01T00:00:00Z')
analysis_period_in_days = [7, 30, 91, 182, 365]
ccys = ['BTCUSD', 'ETHUSD', 'EOSUSD', 'XRPUSD', 'LTCUSD']

###### Winning counts

In [29]:
# From db
data = []
prev_count = None
for period in analysis_period_in_days:
    start = analysis_period_start
    end = start.add(days=period)
    result = get_pricepoints_winner_count(ccys, start, end, analysis_interval, prev_count)
    data.append((period, result))
    prev_result = (start, end, prev_count)


In [ ]:
rows = [list(t[1].values()) for t in data]
df = pd.DataFrame(rows, columns=ccys, index=analysis_period_in_days).round(2)
df['sum'] = df.sum(axis=1)
df = df[ccys].div(df['sum'], axis=0).round(2)
df.index.name = 'Period (days)'
df.head(10)

In [30]:
rows = [list(t[1].values()) for t in data]
df = pd.DataFrame(rows, columns=ccys, index=analysis_period_in_days)

df['sum'] = df.sum(axis=1)
df = df[ccys].div(df['sum'], axis=0).round(2)
df.index.name = 'Period (days)'
df.head(10)

,BTCUSD,ETHUSD,EOSUSD,XRPUSD,LTCUSD
Period (days),,,,,
7,0.31,0.28,0.19,0.08,0.13
30,0.28,0.26,0.18,0.13,0.14
91,0.28,0.25,0.17,0.13,0.16
182,0.23,0.22,0.20,0.17,0.17
365,0.25,0.20,0.22,0.18,0.16


###### No trade counts

In [ ]:
no_trade_list = []
for ccy in ccys:
    prefix = ccy.replace('USD', '')
    column = prefix + '_trades_count'
    trades_count = df[column]
    aggregated_trade_counts = trades_count.value_counts()
    no_trades_count = aggregated_trade_counts.get(0.0)
    if not no_trades_count:
        no_trades_count = 0
        
    ratio = no_trades_count / trades_count.size
        
    no_trade_list.append((prefix, no_trades_count, trades_count.size, ratio))
        
no_trade_df = pd.DataFrame(no_trade_list, columns=['Ccy', 'No Trade Count', 'Total', 'Ratio'])
no_trade_df.head()